### Library import

### Library importing for Lenear Algebra and Data Processing

In [1]:
import numpy as np
import pandas as pd

### Data Visualization

In [2]:
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import datetime

from matplotlib.ticker import FuncFormatter

import warnings
import math as ma
warnings.filterwarnings("ignore")
%matplotlib inline

### Libraries for Algorithm

In [3]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB



### Get Data

In [4]:
customerdemographic= pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name="CustomerDemographic")
transactions= pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name="Transactions")
customeraddress= pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name="CustomerAddress")
newcustomerlist= pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name="NewCustomerList")


FileNotFoundError: [Errno 2] No such file or directory: 'KPMG_VI_New_raw_data_update_final.xlsx'

In [ ]:
customerdemographic.head()

In [ ]:
transactions.head()

In [ ]:
customeraddress.head()

###  Exploring Data Analysis(EDA)

In [ ]:
customerdemographic.info()

In [ ]:
customerdemographic.describe()

### Renaming Column Headers

In [ ]:
list(customerdemographic)

In [ ]:
customerdemographic=customerdemographic.rename(columns={'first_name':'f_name','last_name':'l_name','past_3_years_bike_related_purchases':'past_bike_purchases','owns_car':'car_owner','job_industry_category':'job_idustry'})

In [ ]:
list(customerdemographic)

In [ ]:
customerdemographic.head()

In [ ]:
customerdemographic=customerdemographic.iloc[1: ]
customerdemographic

### Missing Value Find

In [ ]:
total= customerdemographic.isnull().sum().sort_values(ascending=False)
percentage_1=customerdemographic.isnull().sum()/customerdemographic.isnull().count()*100
percentage_2=(round(percentage_1, 1).sort_values(ascending=False))
missing_data=pd.concat([total, percentage_2], axis=1, keys=['total', "%"])
missing_data.head(5)


In [ ]:
customerdemographic.head(10)

### Array to fill null values with

In [ ]:
def check_NAN():
    ret = []
    temp = list(customerdemographic)
    for each in temp:
        ret.append(customerdemographic[each].isna().sum())
    return ret

NaN_list = check_NAN()
print(NaN_list, len(NaN_list))

# Now checking Columns

check = []
k     = -1
for i in NaN_list:
    k += 1
    if i > 0:
        check.append(k)
        
# Now checking Unique

def check_unique():
    temp = list(customerdemographic)
    dict = {}
    for i in range(3, 13):
        ct = 0
        if NaN_list[i] > 0:
            ct = len(customerdemographic[temp[i]].unique()) - 1
        else:
            ct = len(customerdemographic[temp[i]].unique())
        dict[temp[i]] = (ct, customerdemographic[temp[i]].unique())
    return dict

### Now to compare Gender Vs. Bike bought

In [ ]:
# 1. F - Female, 2. M- Male, 3. U-Unknown in array

gender = [0,0,0] 
for each in customerdemographic['gender']:
    if each[0] == "F":
        gender[0] += 1
    elif each[0] == "M":
        gender[1] += 1
    else:
        gender[2] += 1

print(gender)

def gender_female(x, pos):
    
    return int(x)

bike_list = [0,0,0]
bike_list[0] += customerdemographic['past_3_years_bike_related_purchases'][customerdemographic['gender']== 'Female'].sum() + customerdemographic['past_3_years_bike_related_purchases'][customerdemographic['gender']== 'F'].sum()
bike_list[1] += customerdemographic['past_3_years_bike_related_purchases'][customerdemographic['gender']== 'Male'].sum() + customerdemographic['past_3_years_bike_related_purchases'][customerdemographic['gender']== 'M'].sum()
bike_list[2] += customerdemographic['past_3_years_bike_related_purchases'][customerdemographic['gender']== 'U'].sum()

grapho = [gender[0], bike_list[0], gender[1], bike_list[1], gender[2], bike_list[2]]
graphl = [gender[0]*10,bike_list[0], gender[1]*10, bike_list[1], gender[2]*10, bike_list[2]]
colorr = (0.3,0.1,0.4,0.6)



### The 'transaction_date' and 'product_first_sold_date' change to date

In [ ]:
#convert date columns from integer to datetime
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], unit='s')
transactions['transaction_date'].head()

#convert date columns from integer to datetime
transactions['product_first_sold_date'] = pd.to_datetime(transactions['product_first_sold_date'])
transactions['product_first_sold_date'].head()

In [ ]:
transactions.head()

In [ ]:
total= transactions.isnull().sum().sort_values(ascending=False)
percentage_1=transactions.isnull().sum()/transactions.isnull().count()*100
percentage_2=(round(percentage_1, 1).sort_values(ascending=False))
missing_data=pd.concat([total, percentage_2], axis=1, keys=['total', "%"])
missing_data.head(5)


In [ ]:
# Duplicate Values

dups = transactions.duplicated()
transactions[dups].sum()

### Create RFM Features for each customers

In [ ]:
df_RFM = transactions.groupby('customer_id').agg({'transaction_date': lambda y: (transactions['transaction_date'].max().date() - y.max().date()).days,
                                        'customer_id': lambda y: len(y.unique()),  
                                        'list_price': lambda y: round(y.sum(),2)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head(10)

### To automate the segmentation we will use 80% quantile for Recency and Monetary

In [ ]:
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM.head()

### Calculate RFM score and sort customers

In [ ]:
# To do the 2x2 Matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'customer_id': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.head()

### Visualize the Value Matrix and explore some key numbers

In [ ]:
#Average Monetary Matrix

df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
print(df_RFM_M)

### Number of Customer Matrix

In [ ]:
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='customer_id')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
print(df_RFM_C)

### Recency Matrix

In [ ]:
df_RFM_R = df_RFM_SUM.pivot(index='M', columns='R', values='Recency')
df_RFM_R= df_RFM_R.reset_index().sort_values(['M'], ascending = False).set_index(['M'])

df_RFM_R

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(df_RFM.Recency, kde=False, rug=True)

plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(df_RFM.Monetary, kde=False, rug=True)

plt.show()

### The distribution of segments using the squarify plot :